In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

import neuro_morpho_toolbox as nmt
ns = nmt.neuron_set('/home/penglab/Documents/Janelia_1000')
_ = ns.ReduceDimUMAP(feature_set="projection_features", n_neighbors=100)


/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/
Loading CCF Atlas data...
Loading time: 0.88
Loading CCF brain structure data...
Loading time: 0.00
Loading...
100 loaded: 2.7s
Invalid number of soma found: 0
QC failed: AA0114.swc
Invalid number of soma found: 0
QC failed: AA0115.swc
200 loaded: 2.0s
300 loaded: 2.6s
400 loaded: 2.8s
Invalid number of soma found: 0
QC failed: AA0472.swc
500 loaded: 2.5s
Invalid number of soma found: 0
QC failed: AA0576.swc
Invalid number of soma found: 0
QC failed: AA0585.swc
Invalid number of soma found: 0
QC failed: AA0589.swc
600 loaded: 2.3s
Invalid number of soma found: 0
QC failed: AA0639.swc
Invalid number of soma found: 0
QC failed: AA0670.swc
Invalid number of soma found: 0
QC failed: AA0672.swc
700 loaded: 2.2s
Invalid number of soma found: 0
QC failed: AA0754.swc
Invalid number of soma found: 0
QC failed: AA0763.swc
800 loaded: 2.7s
900 loaded: 1.9s
QC failed: AA0952.swc
QC failed: AA0964.swc
QC failed: AA0968.swc
QC


## Using axon projection as features
SETTING CLUSTER NUMBER FROM 8 TO 40

### Hierarchy Clustering
For Hierarchy method
* the most suitable parameter is {'L_method': 'centroid', 'L_metric': 'braycurtis', 'criterionH': 'distance', 'depth': 2, 'R': None, 't': 0.9, 'optimal_ordering': False}
* the ARI is 0.258210445492697
* The setting cluster number's limit is satisfied, the final number of cluster is 19

### Kmeans Clustering
For Kmeans method
* the most suitable parameter is {'n_clusters': 14, 'init': 'random', 'n_init': 20, 'max_iter': 300, 'tol': 0.0001, 'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 'algorithm': 'full'}
* the ARI is 0.2129613044735105
* The setting cluster number's limit is satisfied, the final number of cluster is 14

### DBSCAN Clustering
For DBSCAN method
* the most suitable parameter is {'eps': 0.215, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 'leaf_size': 30, 'p': None, 'n_jobs': None}
* the ARI is 0.2812503177620878
* The setting cluster number's limit is satisfied, the final number of cluster is 24

### HDBSCAN Clustering
For HDBSCAN method, 
* the most suitable parameter is {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.8, 'min_samples': 7, 'p': 2, 'algorithm': 'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 'prediction_data': False, 'match_reference_implementation': False}
* the ARI is 0.28287214992477994
* The setting cluster number's limit is satisfied, the final number of cluster is 31

### SNN Clustering
For SNN_community method
* the most suitable parameter is {'knn': 5, 'metric': 'minkowski', 'method': 'FastGreedy'}
* the ARI is 0.22710873358441527
* The setting cluster number's limit is satisfied, the final number of cluster is 31

In [2]:
par_hier =  {'L_method': 'centroid', 'L_metric': 'Euclidean', 'criterionH': 'distance', 'depth': 2, 'R': None,
             't': 0.9, 'optimal_ordering': False}
par_kmeans = {'n_clusters': 14, 'init': 'random', 'n_init': 20, 'max_iter': 300, 'tol': 0.0001, 
              'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None,
              'algorithm': 'full'}

par_dbscan = {'eps': 0.215, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 
              'leaf_size': 30, 'p': None, 'n_jobs': None}

par_hdbscan = {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.8, 'min_samples': 7, 'p': 2, 'algorithm': 
               'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 
               'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 
               'prediction_data': False, 'match_reference_implementation': False}

par_snn = {'knn':5,'metric':'minkowski','method':'FastGreedy'}


In [3]:
def get_clusters(inputUMAP,method='SNN_community',karg_dict={'knn':5, 'metric':'minkowski','method':'FastGreedy'}):
    methods_allowed = ['SNN_community', 'Hierarchy', 'Kmeans', 'DBSCAN', 'HDBSCAN']
    assert method in methods_allowed, "Please set 'method' as one of the following: 'SNN_community', 'Hierarchy', 'Kmeans', 'DBSCAN', 'HDBSCAN'"
    selectedUMAP = inputUMAP.copy()
    if method=='SNN_community':
        #print('Result of SNN_community')
        if 'knn' in karg_dict.keys():
            knn = karg_dict['knn']
        else:
            knn = 5
        if 'metric' in karg_dict.keys():
            metric = karg_dict['metric']
        else:
            metric = 'minkowski'
        if 'method' in karg_dict.keys():
            community_method = karg_dict['method']
        else:
            community_method = 'FastGreedy'
        cur_clusters = nmt.get_clusters_SNN_community(selectedUMAP, knn=knn, metric=metric, method=community_method)
        

    #karg_dict={'L_method':'single','L_metric':'euclidean'.'t':0.9,'criterionH':'inconsistent', depth=2, R=None, monocrit=None}
    if method =='Hierarchy':
        #print('Result of Hierarchy CLustering')
        cur_clusters = nmt.get_clusters_Hierarchy_clustering(selectedUMAP, karg_dict)


    if method =='Kmeans':
        #print('Result of Kmeans')
        cur_clusters = nmt.get_clusters_kmeans_clustering(selectedUMAP, karg_dict)

    if method =='DBSCAN':
        #print('Result of DBSCAN')
        cur_clusters = nmt.get_clusters_dbscan_clustering(selectedUMAP, karg_dict)

    if method =='HDBSCAN':
        #print('Result of HDBSCAN')
        cur_clusters = nmt.get_clusters_hdbscan_clustering(selectedUMAP, karg_dict)
    selectedUMAP.loc[:,'Cluster'] = ['C' + str(i) for i in cur_clusters]
    return selectedUMAP

## function freq_Matrix(fre_M, uniqueDF, cluster_method, para_test, iternum)
* **fre_M** is the square matrix recording the number of co-clustering
* **uniqueDF** is the dataframe with unrepeated parameters
* **cluster_method** can be 'Hierarchy','Kmeans', 'DBSCAN','HDBSCAN','SNN_community'
* **para_test** is the input parameter dictionary for the cluster method
* **iternum** is the number of iteration to generate the coclustering matrix

In [4]:
import random
import ast
from scipy.spatial.distance import pdist, squareform
import numpy as np
import matplotlib as mpl
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
def freq_Matrix(fre_M,uniqueDF,cluster_method, para_test,iternum):
    
    for i in range(iternum):
        umapDF = ns.UMAP.iloc[random.sample(range(0,ns.UMAP.shape[0]), int(ns.UMAP.shape[0]*0.95)),:].copy()
        resultDF = get_clusters(umapDF.copy(),method =cluster_method,karg_dict = para_test)
        Crange, Ccounts = np.unique(resultDF["Cluster"], return_counts = True)
        for iter_C in Crange:
            selected_row = resultDF[resultDF["Cluster"]==iter_C]
            Clist = selected_row.index.tolist()
            for sample_row in Clist:
                for sample_col in Clist:
                    fre_M.loc[sample_row,sample_col] =  fre_M.loc[sample_row,sample_col]+1
        print('****CODE RUNNING'+str(i/iternum)+' % ****')
    return fre_M.copy()

In [5]:
unique_hier = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_hier.xlsx', index_col=0)
unique_kmeans = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_kmeans.xlsx', index_col=0)
unique_dbscan = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_dbscan.xlsx', index_col=0)
unique_hdbscan = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_hdbscan.xlsx', index_col=0)
unique_snn = pd.read_excel('/home/penglab/FeaCal/dataSource/projFea/unique_snn.xlsx', index_col=0)

In [6]:
fre_M_hier = pd.DataFrame(index = ns.UMAP.index, columns = ns.UMAP.index)
fre_M_hier [fre_M_hier.isnull()]=0
fre_M_hier = freq_Matrix(fre_M_hier ,unique_hier,'Hierarchy',par_hier,5000)
fre_M_hier.to_excel('/home/penglab/FeaCal/dataSource/projFea/fre_M_hier.xlsx')





*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0002 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0004 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0006 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0008 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.001 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0012 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE R

****CODE RUNNING0.0128 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.013 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0132 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0134 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0136 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0138 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.014 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0142 % ****




*************Note:**************
Method centroid requires the distance m

****CODE RUNNING0.0258 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.026 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0262 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0264 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0266 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0268 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.027 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0272 % ****




*************Note:**************
Method centroid requires the distance m

****CODE RUNNING0.0388 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.039 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0392 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0394 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0396 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0398 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.04 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0402 % ****




*************Note:**************
Method centroid requires the distance me

****CODE RUNNING0.0518 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.052 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0522 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0524 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0526 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0528 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.053 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.0532 % ****




*************Note:**************
Method centroid requires the distance m

...

****CODE RUNNING0.9358 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.936 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9362 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9364 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9366 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9368 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.937 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9372 % ****




*************Note:**************
Method centroid requires the distance m

****CODE RUNNING0.9488 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.949 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9492 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9494 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9496 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9498 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.95 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9502 % ****




*************Note:**************
Method centroid requires the distance me

****CODE RUNNING0.9618 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.962 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9622 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9624 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9626 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9628 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.963 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9632 % ****




*************Note:**************
Method centroid requires the distance m

****CODE RUNNING0.9748 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.975 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9752 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9754 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9756 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9758 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.976 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9762 % ****




*************Note:**************
Method centroid requires the distance m

****CODE RUNNING0.9878 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.988 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9882 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9884 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9886 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9888 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.989 % ****




*************Note:**************
Method centroid requires the distance metric to be Euclidean
****CODE RUNNING0.9892 % ****




*************Note:**************
Method centroid requires the distance m

SETTING CLUSTER NUMBER FROM 8 TO 40
#### Using soma location as features
### Hierarchy Clustering
For Hierarchy method, the most suitable parameter is {'L_method': 'complete', 'L_metric': 'braycurtis', 'criterionH': 'distance', 'depth': 2, 'R': None, 't': 0.9, 'optimal_ordering': False}
For Hierarchy method, the ARI is 0.3268728951610669
The setting cluster number's limit is satisfied, the final number of cluster is 9
### Kmeans Clustering
For Kmeans method, the most suitable parameter is {'n_clusters': 10, 'init': 'random', 'n_init': 7, 'max_iter': 300, 'tol': 0.0001, 'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 'algorithm': 'auto'}
For Kmeans method, the ARI is 0.18870810160144874
The setting cluster number's limit is satisfied, the final number of cluster is 10
### DBSCAN Clustering
For DBSCAN method, the most suitable parameter is {'eps': 0.9, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 'leaf_size': 30, 'p': None, 'n_jobs': None}
For DBSCAN method, the ARI is 0.2630886481583674
The setting cluster number's limit is satisfied, the final number of cluster is 29
### HDBSCAN Clustering
For HDBSCAN method, the most suitable parameter is {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.9, 'min_samples': 9, 'p': 2, 'algorithm': 'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 'prediction_data': False, 'match_reference_implementation': False}
For HDBSCAN method, the ARI is 0.26100195542904575
The setting cluster number's limit is satisfied, the final number of cluster is 34


### SNN Clustering
For SNN_community method, the most suitable parameter is {'knn': 5, 'metric': 'minkowski', 'method': 'FastGreedy'}
For SNN_community method, the ARI is 0.13064660013981225
The setting cluster number's limit is not satisfied, the final number of cluster is 65

In [7]:
ns = nmt.neuron_set('/home/penglab/Documents/Janelia_1000')
_ = ns.ReduceDimUMAP(feature_set="soma_features", n_neighbors=100)

Loading...
100 loaded: 2.3s
Invalid number of soma found: 0
QC failed: AA0114.swc
Invalid number of soma found: 0
QC failed: AA0115.swc
200 loaded: 2.2s
300 loaded: 2.4s
400 loaded: 2.4s
Invalid number of soma found: 0
QC failed: AA0472.swc
500 loaded: 2.2s
Invalid number of soma found: 0
QC failed: AA0576.swc
Invalid number of soma found: 0
QC failed: AA0585.swc
Invalid number of soma found: 0
QC failed: AA0589.swc
600 loaded: 2.3s
Invalid number of soma found: 0
QC failed: AA0639.swc
Invalid number of soma found: 0
QC failed: AA0670.swc
Invalid number of soma found: 0
QC failed: AA0672.swc
700 loaded: 2.0s
Invalid number of soma found: 0
QC failed: AA0754.swc
Invalid number of soma found: 0
QC failed: AA0763.swc
800 loaded: 2.6s
900 loaded: 1.7s
QC failed: AA0952.swc
QC failed: AA0964.swc
QC failed: AA0968.swc
QC failed: AA0977.swc
Finding soma locations...
Getting projection features...
Number of input neurons: 987
Number of input features: 632
Getting metadata...


In [8]:
par_hier2 =  {'L_method': 'complete', 'L_metric': 'braycurtis', 'criterionH': 'distance', 'depth': 2, 'R': None, 
             't': 0.9, 'optimal_ordering': False} 
par_kmeans2 = {'n_clusters': 10, 'init': 'random', 'n_init': 7, 'max_iter': 300, 'tol': 0.0001, 
              'precompute_distances': 'auto', 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 
              'algorithm': 'auto'} 

par_dbscan2 = {'eps': 0.9, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 
              'leaf_size': 30, 'p': None, 'n_jobs': None}

par_hdbscan2 =  {'min_cluster_size': 5, 'metric': 'euclidean', 'alpha': 0.9, 'min_samples': 9, 'p': 2, 'algorithm': 
                'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 
                'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 
                'prediction_data': False, 'match_reference_implementation': False} 

par_snn2 = {'knn':5,'metric':'minkowski','method':'FastGreedy'}


In [ ]:
fre_M_hier2 = pd.DataFrame(index = ns.UMAP.index, columns = ns.UMAP.index)
fre_M_hier2 [fre_M_hier2.isnull()]=0
fre_M_hier2 = freq_Matrix(fre_M_hier2 ,unique_hier,'Hierarchy',par_hier2,5000)
fre_M_hier2.to_excel('/home/penglab/FeaCal/dataSource/somaLoc/fre_M_hier.xlsx')

****CODE RUNNING0.0 % ****
****CODE RUNNING0.0002 % ****
****CODE RUNNING0.0004 % ****
****CODE RUNNING0.0006 % ****
****CODE RUNNING0.0008 % ****
****CODE RUNNING0.001 % ****
****CODE RUNNING0.0012 % ****
****CODE RUNNING0.0014 % ****
****CODE RUNNING0.0016 % ****
****CODE RUNNING0.0018 % ****
****CODE RUNNING0.002 % ****
****CODE RUNNING0.0022 % ****
****CODE RUNNING0.0024 % ****
****CODE RUNNING0.0026 % ****
****CODE RUNNING0.0028 % ****
****CODE RUNNING0.003 % ****
****CODE RUNNING0.0032 % ****
****CODE RUNNING0.0034 % ****
****CODE RUNNING0.0036 % ****
****CODE RUNNING0.0038 % ****
****CODE RUNNING0.004 % ****
****CODE RUNNING0.0042 % ****
****CODE RUNNING0.0044 % ****
****CODE RUNNING0.0046 % ****
****CODE RUNNING0.0048 % ****
****CODE RUNNING0.005 % ****
****CODE RUNNING0.0052 % ****
****CODE RUNNING0.0054 % ****
****CODE RUNNING0.0056 % ****
****CODE RUNNING0.0058 % ****
****CODE RUNNING0.006 % ****
****CODE RUNNING0.0062 % ****
****CODE RUNNING0.0064 % ****
****CODE RUNNING0.0

****CODE RUNNING0.0552 % ****
****CODE RUNNING0.0554 % ****
****CODE RUNNING0.0556 % ****
****CODE RUNNING0.0558 % ****
****CODE RUNNING0.056 % ****
****CODE RUNNING0.0562 % ****
****CODE RUNNING0.0564 % ****
****CODE RUNNING0.0566 % ****
****CODE RUNNING0.0568 % ****
****CODE RUNNING0.057 % ****
****CODE RUNNING0.0572 % ****
****CODE RUNNING0.0574 % ****
****CODE RUNNING0.0576 % ****
****CODE RUNNING0.0578 % ****
****CODE RUNNING0.058 % ****
****CODE RUNNING0.0582 % ****
****CODE RUNNING0.0584 % ****
****CODE RUNNING0.0586 % ****
****CODE RUNNING0.0588 % ****
****CODE RUNNING0.059 % ****
****CODE RUNNING0.0592 % ****
****CODE RUNNING0.0594 % ****
****CODE RUNNING0.0596 % ****
****CODE RUNNING0.0598 % ****
****CODE RUNNING0.06 % ****
****CODE RUNNING0.0602 % ****
****CODE RUNNING0.0604 % ****
****CODE RUNNING0.0606 % ****
****CODE RUNNING0.0608 % ****
****CODE RUNNING0.061 % ****
****CODE RUNNING0.0612 % ****
****CODE RUNNING0.0614 % ****
****CODE RUNNING0.0616 % ****
****CODE RUNNING0

****CODE RUNNING0.1104 % ****
****CODE RUNNING0.1106 % ****
****CODE RUNNING0.1108 % ****
****CODE RUNNING0.111 % ****
****CODE RUNNING0.1112 % ****
****CODE RUNNING0.1114 % ****
****CODE RUNNING0.1116 % ****
****CODE RUNNING0.1118 % ****
****CODE RUNNING0.112 % ****
****CODE RUNNING0.1122 % ****
****CODE RUNNING0.1124 % ****
****CODE RUNNING0.1126 % ****
****CODE RUNNING0.1128 % ****
****CODE RUNNING0.113 % ****
****CODE RUNNING0.1132 % ****
****CODE RUNNING0.1134 % ****
****CODE RUNNING0.1136 % ****
****CODE RUNNING0.1138 % ****
****CODE RUNNING0.114 % ****
****CODE RUNNING0.1142 % ****
****CODE RUNNING0.1144 % ****
****CODE RUNNING0.1146 % ****
****CODE RUNNING0.1148 % ****
****CODE RUNNING0.115 % ****
****CODE RUNNING0.1152 % ****
****CODE RUNNING0.1154 % ****
****CODE RUNNING0.1156 % ****
****CODE RUNNING0.1158 % ****
****CODE RUNNING0.116 % ****
****CODE RUNNING0.1162 % ****
****CODE RUNNING0.1164 % ****
****CODE RUNNING0.1166 % ****
****CODE RUNNING0.1168 % ****
****CODE RUNNING

****CODE RUNNING0.1656 % ****
****CODE RUNNING0.1658 % ****
****CODE RUNNING0.166 % ****
****CODE RUNNING0.1662 % ****
****CODE RUNNING0.1664 % ****
****CODE RUNNING0.1666 % ****
****CODE RUNNING0.1668 % ****
****CODE RUNNING0.167 % ****
****CODE RUNNING0.1672 % ****
****CODE RUNNING0.1674 % ****
****CODE RUNNING0.1676 % ****
****CODE RUNNING0.1678 % ****
****CODE RUNNING0.168 % ****
****CODE RUNNING0.1682 % ****
****CODE RUNNING0.1684 % ****
****CODE RUNNING0.1686 % ****
****CODE RUNNING0.1688 % ****
****CODE RUNNING0.169 % ****
****CODE RUNNING0.1692 % ****
****CODE RUNNING0.1694 % ****
****CODE RUNNING0.1696 % ****
****CODE RUNNING0.1698 % ****
****CODE RUNNING0.17 % ****
****CODE RUNNING0.1702 % ****
****CODE RUNNING0.1704 % ****
****CODE RUNNING0.1706 % ****
****CODE RUNNING0.1708 % ****
****CODE RUNNING0.171 % ****
****CODE RUNNING0.1712 % ****
****CODE RUNNING0.1714 % ****
****CODE RUNNING0.1716 % ****
****CODE RUNNING0.1718 % ****
****CODE RUNNING0.172 % ****
****CODE RUNNING0.